In [92]:
import re

str = u'陈奕迅演唱(十年)、(浮夸)、(不要说话)'
string = re.finditer('\((.*?)\)', str)
print(string)

In [1]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

In [2]:
from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *

In [3]:
#%%bash
#pip install pytorch-pretrained-bert

In [4]:
seed = 42#default 42
np.random.seed(seed)
random.seed(seed)

In [5]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

config = Config(
    testing=False,
    bert_model_name="bert-base-uncased",
    max_lr=3e-5,
    epochs=7,
    use_fp16=True,
    bs=32,
    discriminative=False,
    max_seq_len=500,
)

In [69]:
from sklearn.model_selection import train_test_split

DATA_ROOT = Path("") 

train, test = [pd.read_csv(DATA_ROOT / fname) for fname in ["task2_trainset.csv", "task2_public_testset.csv"]]

In [70]:
train.drop('Title',axis=1,inplace=True)
train.drop('Categories',axis=1,inplace=True)
test.drop('Title',axis=1,inplace=True)
test.drop('Categories',axis=1,inplace=True)
####################################################
train.drop('Created Date',axis=1, inplace=True)
train.drop('Authors',axis=1,inplace=True)
test.drop('Created Date',axis=1, inplace=True)
test.drop('Authors',axis=1,inplace=True)

In [71]:
train['THEORETICAL']=0
train['ENGINEERING']=0
train['EMPIRICAL']=0
train['OTHERS']=0
for i in range(train.Id.count()):
    string = train['Task 2'][i]
    train['THEORETICAL'][i] = 1 if (string.find('THEORETICAL')!=-1) else 0
    train['ENGINEERING'][i] = 1 if (string.find('ENGINEERING')!=-1) else 0
    train['EMPIRICAL'][i] = 1 if (string.find('EMPIRICAL')!=-1) else 0
    train['OTHERS'][i] = 1 if (string.find('OTHERS')!=-1) else 0   
train.head()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:

,Id,Abstract,Task 2,THEORETICAL,ENGINEERING,EMPIRICAL,OTHERS
0,D00001,Rapid popularity of Internet of Things (IoT) a...,THEORETICAL,1,0,0,0
1,D00002,"In this paper, we address the problem of compu...",THEORETICAL,1,0,0,0
2,D00003,High quality upsampling of sparse 3D point clo...,ENGINEERING,0,1,0,0
3,D00004,Internet is the main source of information now...,EMPIRICAL,0,0,1,0
4,D00005,Automated Facial Expression Recognition (FER) ...,ENGINEERING,0,1,0,0


In [72]:
train.drop('Task 2',axis=1,inplace=True)
train

,Id,Abstract,THEORETICAL,ENGINEERING,EMPIRICAL,OTHERS
0,D00001,Rapid popularity of Internet of Things (IoT) a...,1,0,0,0
1,D00002,"In this paper, we address the problem of compu...",1,0,0,0
2,D00003,High quality upsampling of sparse 3D point clo...,0,1,0,0
3,D00004,Internet is the main source of information now...,0,0,1,0
4,D00005,Automated Facial Expression Recognition (FER) ...,0,1,0,0
...,...,...,...,...,...,...
6995,D06996,We have witnessed the discovery of many techni...,1,1,0,0
6996,D06997,State-of-the-art slot filling models for goal-...,0,1,0,0
6997,D06998,Real-time semantic segmentation plays an impor...,0,1,0,0
6998,D06999,We propose a neural embedding algorithm called...,1,0,1,0


In [73]:
train.head()

,Id,Abstract,THEORETICAL,ENGINEERING,EMPIRICAL,OTHERS
0,D00001,Rapid popularity of Internet of Things (IoT) a...,1,0,0,0
1,D00002,"In this paper, we address the problem of compu...",1,0,0,0
2,D00003,High quality upsampling of sparse 3D point clo...,0,1,0,0
3,D00004,Internet is the main source of information now...,0,0,1,0
4,D00005,Automated Facial Expression Recognition (FER) ...,0,1,0,0


In [74]:
print(train.THEORETICAL.value_counts()[1] / len(train))
print(train.ENGINEERING.value_counts()[1] / len(train))
print(train.EMPIRICAL.value_counts()[1] / len(train))
print(train.OTHERS.value_counts()[1] / len(train))

0.45971428571428574
0.48442857142857143
0.3057142857142857
0.037


In [75]:
max_sent = max([len(train['Abstract']) for data in train])
max_sent

7000

In [80]:
for i in range(train.Id.count()):
    train['Abstract'][i]=train['Abstract'][i].replace("$$$", " ")
    train['Abstract'][i]=train['Abstract'][i].replace(".", " ")
    train['Abstract'][i]=train['Abstract'][i].replace(":", " ")
    train['Abstract'][i]=train['Abstract'][i].replace("%", "percentage")
    train['Abstract'][i]=train['Abstract'][i].replace("%", "in other word")


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd

In [81]:
for i in range(test.Id.count()):
    test['Abstract'][i]=test['Abstract'][i].replace("$$$", " ")
    test['Abstract'][i]=test['Abstract'][i].replace(".", " ")
    test['Abstract'][i]=test['Abstract'][i].replace(":", " ")
    test['Abstract'][i]=test['Abstract'][i].replace("%", "percentage")
    test['Abstract'][i]=test['Abstract'][i].replace("%", "in other word")


In [82]:
train.head()

,Id,Abstract,THEORETICAL,ENGINEERING,EMPIRICAL,OTHERS
0,D00001,Rapid popularity of Internet of Things (IoT) a...,1,0,0,0
1,D00002,"In this paper, we address the problem of compu...",1,0,0,0
2,D00003,High quality upsampling of sparse 3D point clo...,0,1,0,0
3,D00004,Internet is the main source of information now...,0,0,1,0
4,D00005,Automated Facial Expression Recognition (FER) ...,0,1,0,0


In [79]:
train['Abstract'][6999]

'Compute-and-forward (CAF) relaying is effective to increase bandwidth efficiency of wireless two-way relay channels  In a CAF scheme, a relay is designed to decode a linear combination composed of transmitted messages from other terminals or relays  Design for error-correcting codes and its decoding algorithms suitable for CAF relaying schemes remain as an important issue to be studied  As described in this paper, we will present an asymptotic performance analysis of LDPC codes over two-way relay channels based on density evolution (DE)  Because of the asymmetric characteristics of the channel, we use the population dynamics DE combined with DE formulas for asymmetric channels to obtain BP thresholds  Additionally, we also evaluate the asymptotic performance of spatially coupled LDPC codes for two-way relay channels  The results indicate that the spatial coupling codes yield improvements in the BP threshold compared with corresponding uncoupled codes for two-way relay channels  Finall